In [ ]:
import IPython.display as ipd
import os
import sys

os.environ["MKL_NUM_THREADS"]='1'
os.environ["NUMEXPR_NUM_THREADS"]='1'
os.environ["OMP_NUM_THREADS"]='1'

import torch
import numpy as np
import librosa

sys.path.insert(0, "../")
from utils import get_hparams
from models import get_wrapper

os.environ["CUDA_VISIBLE_DEVICES"]='2'
device = 'cpu'     # 'cpu' or 'cuda'

# Load Model

In [ ]:
base_dir = "../logs/hilcodec_music"
hps = get_hparams(f"{base_dir}/config.yaml", base_dir)
wrapper = get_wrapper(hps.model)(hps, device=device)
wrapper.load()
wrapper.eval()

sr = hps.data.sampling_rate
hop_size = wrapper.hop_size
lookahead = getattr(hps.train, "lookahead", 0)

In [ ]:
wav, _ = librosa.load("../onnx/input_speech.wav", sr=sr)
wav_r = torch.from_numpy(wav).view(1, 1, -1).to(device=device).clamp(min=-1, max=1)
wav_len = wav_r.size(-1) // hop_size * hop_size
wav_r = wav_r[:, :, :wav_len]
ipd.display(ipd.Audio(wav_r.view(-1).numpy(), normalize=False, rate=sr))

with torch.no_grad():
    wav_g, *_ = wrapper.model(wav_r, n=8)
ipd.display(ipd.Audio(wav_g.view(-1).numpy(), normalize=False, rate=sr))